# Assignment 2: Content-based Filtering

## Reading Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
tags = pd.read_csv('data/movie-tags.csv', header=None, names=['movie_id', 'tag'])
movies = pd.read_csv('data/movie-titles.csv', header=None, names=['movie_id', 'movie', 'genres'])
ratings = pd.read_csv('data/ratings.csv', header=None, names=['user_id', 'movie_id', 'rating'])

## Part 1: TF-IDF Recommender with Unweighted Profiles

### Compute item-tag vectors (the model)

Step 1: Iterate through items, building the term vector 𝒒𝑖 for each item and a global document frequency vector 𝒅. At this stage, these are unnormalized term and document frequency vectors, storing the number of times the term appears on each document or the number of documents in which it appears.

* first initializing item_tag_dict

In [3]:
item_tag_dict = {}
for movie in movies['movie_id'].tolist():
    item_tag_dict[movie] = {}

* then, filling item_tag_dict

In [4]:
item_tags = tags.groupby('movie_id')
for item in item_tags:
    item_i_tags = item[1]['tag'].value_counts().to_dict()
    item_tag_dict[item[0]] = item_i_tags
Q = pd.DataFrame.from_dict(item_tag_dict, orient='index')

* also, we need to put items with no tag count in Q, and sort it based on item_id

In [5]:
item_with_no_tags = [i for i in item_tag_dict.keys() if len(item_tag_dict[i]) == 0]
for item in item_with_no_tags:
    Q.loc[item] = np.nan
Q = Q.sort_index()

Step 2: Iterate through each item again, performing the following:<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a. Divide each term value 𝑞̂𝑖𝑡 by the log of the document frequency (𝑙𝑛 𝑑𝑡). The resulting
vector 𝒒𝑖 is the TF-IDF vector.

In [6]:
Q = Q.fillna(0)
df = Q.count()
idf = np.log(df)
Q = Q/idf

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;b. After dividing each term value by the log of the DF, compute the length (Euclidean norm) of the TF-IDF vector 𝒒𝑖, and divide each element of it by the length to yield a unit vector 𝒒𝑖. 

In [7]:
Q = Q.div(np.sqrt(np.square(Q).sum(axis=1)), axis=0)
Q = Q.fillna(0)

### Build user profile for each query user

The profile is the sum of the item-tag vectors of all items the user has rated positively (>= 3.5 stars)

$$ p_{ut} = \sum_{i\in{I},\, r_{ui} \geq{3.5}}{q_{it}} $$

* first building the user rating matrix (movies * users)

In [8]:
item_rating_dict = {}
item_ratings = ratings.groupby('movie_id')
for item in item_ratings:
    item_i_ratings = item[1][['user_id','rating']].set_index('user_id').to_dict()['rating']
    item_rating_dict[item[0]] = item_i_ratings
R = pd.DataFrame.from_dict(item_rating_dict, orient='index')
R = R.sort_index()
R = R.T

* then, converting ratings to 1 if ratings>=3.5 otherwise 0

In [9]:
R35 = R.copy()
R35[R35<3.5]=0
R35[R35>=3.5]=1
R35 = R35.fillna(0)

* finally, doing the dot product

In [10]:
P = R35.dot(Q)

### Generate item scores for each user

The score for an item is the cosine between that item’s tag vector and the user’s profile vector.<br />
Cosine similarity is defined as follows:

$$ \cos{(p_{u},\,q_{i})} = \frac{p_{u} \cdot q_{i}}{\parallel p_{u} \parallel_{2} \parallel q_{i} \parallel_{2}} = \frac{\sum_{t}{q_{ut}p_{ut}}}{\sqrt{\sum_{t}{q_{ut}^2}}\sqrt{\sum_{t}{p_{ut}^2}}} $$

In [11]:
from scipy.spatial.distance import cdist

* pairwise 'cosine' distance

In [12]:
c = cdist(P.values, Q.values, 'cosine')

* cosine similarity score matrix (user * item)

In [13]:
S = pd.DataFrame(1 - c, index=P.index, columns=Q.index)

## Part 2: Weighted User Profile

In this variant, rather than just summing the vectors for all positively-rated items, compute a weighted sum of the item vectors for all rated items, with weights being based on the user’s rating:

$$ p_{u} = \sum_{i\in{I(u)}}{ (r_{ui} - \mu_{u}) \cdot q_{i} } $$

In [72]:
RW = R.copy()
mu = RW.mean(axis=1)
W = RW.sub(s, axis=0)
W = W.fillna(0)

In [74]:
PW = W.dot(Q)

In [75]:
PW

family    classic  humorous   fantasy    bright  fanciful      toys  \
12288   0.680782   2.239962  0.889096  5.076983 -0.190889  0.401534  0.104936   
71680   0.915897   2.721329 -0.317350 -2.257227  0.046179  0.009585  0.024708   
32780  -2.389675  11.561317  1.349053  2.374417 -0.552643 -0.052796 -0.008220   
106512  0.320252   5.650538  0.767279  0.084404  0.065380  0.352630 -0.070522   
32785   0.335856   2.114862 -0.751138 -3.016836  0.024910  0.192757  0.099642   
67615   0.412273   1.047495  0.273987  0.534646  0.067169  0.029938  0.360142   
59424   0.793354   0.113296 -0.710635  1.758995  0.034288  0.144044  0.124346   
96296   0.263022  -0.311171  0.347092  0.041191  0.031587 -0.106285 -0.126373   
106537 -0.385471  -0.259102  0.125696  1.947083  0.051618  0.271443  0.112651   
57387   0.032051   1.277970  0.211503 -0.845837  0.037970  0.025043  0.033007   
38957   1.014379   3.102946  0.142301  1.291661 -0.054084  0.101823 -0.184665   
47150  -0.137610  -0.801852 -0.385811  0.611905  0.212757  0.397232  0.303865   
20530   0.578715   2.773178 -0.182872 -0.216013  0.127689  0.336400  0.327810   
102750 -1.344543   0.477915 -0.167846  0.314474  0.035433  0.019139 -0.050523   
129078 -0.334990   5.175728  0.641086  0.001236  0.085750  0.322902  0.604701   
104508 -0.320043   1.466429 -0.036804  1.120982 -0.302938  0.081798  0.090336   
63551  -0.000401   0.825078 -0.399835  1.206116  0.027752  0.092914 -0.022349   
96320   0.106962   1.459339  0.010591  0.292292  0.094736  0.258339  0.134005   
124998  0.760982   7.459156  1.370549  5.261194  0.097166  1.052409  0.176580   
112736  0.139293   0.363774 -0.024975 -0.267585  0.012282  0.012282  0.049126   
14437   0.005820   3.528142  0.340338  1.901085 -0.078293  0.110993  0.008578   
90216   0.372612   0.772460  0.029898 -0.384009 -0.050077  0.066179  0.092665   
359     0.895365   7.430051 -0.066993  1.818018  0.066612  0.318989  0.468697   
120941 -0.179329  -0.145481  0.116129 -2.599028 -0.010181 -0.151340  0.223126   
65648   0.089555   0.134293  0.023634  0.323964  0.024303  0.057598  0.097210   
96370  -0.947930   8.735331  2.177640  2.929975  0.423490  0.576447  0.365963   
4215    1.172732   3.054415  0.153830 -1.192928  0.139138  0.215861  0.398699   
63615   0.057889   0.570184  0.425619  0.456724  0.037831  0.261774 -0.156027   
60971  -0.812678   0.328425  0.125014 -0.038521  0.021749  0.055869  0.297304   
102532  0.535958   6.734530  2.843067  2.333457  0.196774  0.243784  0.274111   
...          ...        ...       ...       ...       ...       ...       ...   
69374   0.245940   0.214087 -0.124101 -0.694190  0.000000 -0.133125  0.000000   
27142   0.000000  -0.040131 -0.107466  0.257145 -0.013792  0.000000  0.000000   
45854   0.063116   0.811978 -0.034533  0.146132 -0.033181 -0.022730  0.000000   
93203   0.024678   0.126956  0.168805  1.190169  0.000000  0.146942  0.000000   
66756   0.036809  -0.508884 -0.170272 -0.349752  0.000000  0.000000  0.000000   
126292 -0.126156   0.499383 -0.017911  0.154804  0.000000  0.089085  0.000000   
109970 -0.230965  -0.455000  0.517844  1.943883  0.000000 -0.187963  0.000000   
97436  -0.116119  -0.295009  0.131559  0.000000  0.000000  0.000000  0.000000   
12014   0.345041   1.138986 -0.494074  0.132550 -0.199265 -0.298542 -0.199265   
111036  0.000000   0.477198  0.000000 -0.704394  0.000000  0.000000  0.000000   
104529  0.053318   0.478780 -0.110130  0.182157  0.000000 -0.048245  0.000000   
127647  0.007059   0.142867  0.209642  1.880344  0.037306  0.176859  0.000000   
124981  0.000000   0.641789  0.036646  0.000000  0.000000  0.000000  0.000000   
121403  0.000000   0.141902  0.026233  0.005459  0.015314  0.000000  0.000000   
17194   0.000000   0.544392  0.101543  0.137965  0.000000  0.041392  0.000000   
18204  -1.877281   0.081580  0.096779  0.728395  0.000000  0.054960  0.000000   
12255   0.000000   0.221216  0.336143  0.077776  0.000000  0.161249  0.000000   
119134  0.196744   0.636841  0.000000  2.